In [ ]:
'''
Run multiple classification models on the VGGish extracted features. 
To use, just change the source directory of the metal features file.
Use this after running the extract_features_from_metal.py file in the vggish_feature_extraction folder.
'''

In [1]:
import pandas as pd
import os

os.chdir('C:\\Users\\blake\\OneDrive - The University of Texas at Austin')

metal_features = pd.read_csv('vggish_metal_features.csv')
print(metal_features)

                                                   song        genre   
0      1. The Hollow - a perfect circle-avgiqNapUx0.wav  alternative  \
1     10 Years - Beautiful (Official Video)-C6iQpkkW...  alternative   
2                     10 Years - Fix Me-DCbkeUxkbYc.wav  alternative   
3               10 Years - Shoot It Out-3VCvSJo-yC0.wav  alternative   
4             10 Years - Waking Up [HD]-kHg_XwPuY2E.wav  alternative   
...                                                 ...          ...   
2002  Wehrmacht - You Broke My Heart, So I Broke You...       thrash   
2003   Whiplash - Power Thrashing Death-IVX4tumzsn0.wav       thrash   
2004        Whiplash-Burning Of Atlanta-ByUD7JemXzU.wav       thrash   
2005      XENTRIX - For Whose Advantage-_0M9Tl1fx4Y.wav       thrash   
2006  𝗖𝗔𝗥𝗖𝗔𝗦𝗦 ''1985 † Thrasher's Abattoir'' [Melo D...       thrash   

      Feature 0  Feature 1  Feature 2  Feature 3  Feature 4  Feature 5   
0     -0.228171  -0.023990  -0.124108   0.326429  -0.028174  

In [2]:
print(metal_features.shape)
metal_features = metal_features.dropna()
print(metal_features.shape)

(2007, 16384)
(1995, 16384)


In [3]:
metal_features = metal_features.drop_duplicates(subset='song', keep="first")
print(metal_features.shape)

(1947, 16384)


In [4]:
import tensorflow as tf
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [5]:
def train_CNN_model(X_train, Y_train, X_test, Y_test):
    
    # Define the architecture of the CNN
    model=tf.keras.models.Sequential([
        layers.Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], 1)),
        layers.MaxPooling1D(2),
        layers.Conv1D(32, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(11, activation='softmax')   
    ])
    # Compile the model
    model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, Y_train, epochs=100, batch_size=16, validation_data=(X_test, Y_test))

    return model

def train_mlp_model(X_train, Y_train, X_test, Y_test):
    mlp = MLPClassifier(hidden_layer_sizes=(100, 100, 100), max_iter=500, alpha=0.001,solver='sgd', verbose=0,  random_state=21,tol=0.000000001, activation='relu', learning_rate='adaptive')
    mlp.fit(X_train, Y_train)
    return mlp

def train_logistic_model(X_train, Y_train, X_test, Y_test):
    logreg = LogisticRegression(max_iter=500)
    logreg.fit(X_train, Y_train)
    return logreg

def train_random_forest_model(X_train, y_train, X_test, y_test):
    clf = RandomForestClassifier(n_estimators = 1000)  
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print("ACCURACY OF THE MODEL: ", accuracy_score(y_test, y_pred))
    return clf
    
def calculate_accuracy(model, X, Y):
    accuracy = accuracy_score(Y, model.predict(X))
    return accuracy

In [6]:
X = metal_features.drop(columns=['song', 'genre'])
y = metal_features['genre']


In [7]:
def label_to_int(label):
    if label == 'alternative':
        return 0
    elif label == 'death':
        return 1
    elif label == 'folk':
        return 2
    elif label == 'glam':
        return 3
    elif label == 'industrial':
        return 4
    elif label == 'metalcore':
        return 5
    elif label == 'nu':
        return 6
    elif label == 'NWOBHM':
        return 7
    elif label == 'progressive':
        return 8
    elif label == 'symphonic':
        return 9
    elif label == 'thrash':
        return 10
    
y = y.apply(lambda x: label_to_int(x))
print(y)

0        0
1        0
2        0
3        0
4        0
        ..
2001    10
2003    10
2004    10
2005    10
2006    10
Name: genre, Length: 1947, dtype: int64


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)


In [9]:
# cnn_model = train_CNN_model(X_train, y_train, X_test, y_test)
mlp_model = train_mlp_model(X_train, y_train, X_test, y_test)

c:\Users\blake\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [10]:
print(calculate_accuracy(mlp_model, X_test, y_test))

0.6652977412731006


In [76]:
rf_model = train_random_forest_model(X_train, y_train, X_test, y_test)

ACCURACY OF THE MODEL:  0.5626283367556468
